# Tensor Class
<h3> This class is used by autogradient python file which is currently under construction </h3>

In [1]:
class Tensor:
    """
    Refer The below link to know why mutable datasetrutures cant be in the initialization
    https://stackoverflow.com/questions/4841782/python-constructor-and-default-value
    """
    def __init__(self,value,parent=None,child=None,operation=None,grad=None,gradstatus =0):
        self.value = value
        self.grad = grad
        self.operation = operation
        self.gradstatus = 0
        if child is None:
            self.child = []
        else:
            self.child = child
        if parent is None:
            self.parent = []
        else:
            self.parent = parent
            
        
    def get_value(self):
        return self.value
    def get_consumer(self):
        return self.child
    def get_parent(self):
        return self.parent
    def get_operation(self):
        return self.operation
    def get_grad(self):
        return self.grad
    def get_gradstatus(self):
        return self.gradstatus
    
    def update_child(self,child):
        self.child.append(child)
    def update_value(self,value):
        self.value = value
    def update_grad(self,grad):
        self.grad += grad
   
    def set_grad(self,grad):
        self.grad = grad
    def set_gradstatus(self,stat):
        self.gradstatus = stat
        
    def size(self):
        return 1
    def __repr__(self):
        return 'tensor object:'+ str(id(self)) 

# Import guidance
Before attempting the below imports of im2col3, im2col4, ..... You should type in the below code in the command line opened in the folder being the same as the folder holding these code files.

> python setup.py build_ext --inplace

In [11]:
import im2col3
import im2col4 # returns 2d
import im2col2 
import col2im
import im2col
import im2col6 #from cs231n

class Convolution2D:
    import numpy as np
    def __init__(self,stride=2,pad=1):
        self.stride = stride
        self.pad = pad
        
    def __call__(self,X,w):
        
        res = Convolution2D.__conv_v5_3d__(X,w,self.stride,self.pad)
        return res
    
    
            
    def __get_input_value__(input1,input2):
        return (input1.get_value(),input2.get_value())
    
    def __pad__(X_val,pad):
        if len(X_val.shape)==3:
            return np.pad(X_val,((0,0),(pad,pad),(pad,pad)),'constant',constant_values=0)
        elif len(X_val.shape)==4:
            return np.pad(X_val,((0,0),(0,0),(pad,pad),(pad,pad)),'constant',constant_values=0)
    def __get_shape__(X_val,w_val):
        
        #l,m,n = X_val.shape # 3d array:no of channels; m,n - row,col
        #o,p,q = w_val.shape # 3d array:no of filters; p,q - row,col
        return (X_val.shape,w_val.shape)
    def __assert_shape__(x_shp,w_shp,stride):
        if len(x_shp)==3:
            try:
                assert(x_shp[0]==w_shp[0]), 'Wrong channel,filter'
                assert((x_shp[1]-w_shp[1])%stride == 0), 'Wrong stride'
                assert((x_shp[2]-w_shp[2])%stride == 0), 'Wrong stride'
                return (int((x_shp[1]-w_shp[1])/stride+1),int((x_shp[2]-w_shp[2])/stride+1))
            except:
                print('Error: Incompatible Tensor dimensions for convolution')
                return 0
        if len(x_shp)==4:
            try:
                assert(x_shp[1]==w_shp[1]), 'Wrong channel,filter'
                assert((x_shp[2]-w_shp[2])%stride == 0), 'Wrong stride'
                assert((x_shp[3]-w_shp[3])%stride == 0), 'Wrong stride'
                return (x_shp[0],w_shp[0],int((x_shp[2]-w_shp[2])/stride+1),int((x_shp[3]-w_shp[3])/stride+1))
            except:
                print('Error: Incompatible Tensor dimensions for convolution')
                return 0

    def __rect_shape__(X_val,w_val):
        if(len(X_val.shape)==2):
            X_val.reshape(1,X_val.shape)
        if(len(w_val.shape)==2):
            w_val.reshape(1,w_val.shape)
        return X_val,w_val
    
    
    def __conv_v2_3d__(X,w,stride,pad):
        X_val,w_val  = Convolution2D.__get_input_value__(X,w)
        X_val        = Convolution2D.__pad__( X_val ,pad)
        x_shp,w_shp  = Convolution2D.__get_shape__(X_val,w_val)
        res_shape    = Convolution2D. __assert_shape__(x_shp,w_shp,stride)
        X_res        = np.zeros(shape = res_shape)
        if res_shape:
            for k in range(w_shp[0]):
                for i in range(w_shp[2]):
                    for j in range(w_shp[3]):
                        for l in range(w_shp[1]):

                            row_idx = np.arange(i,x_shp[2]-w_shp[2]+i+1,stride)
                            col_idx = np.arange(j,x_shp[3]-w_shp[3]+j+1,stride)
                            X_res[:,k,:,:] += X_val[:,l,row_idx[:, None], col_idx]*w_val[k,l,i,j] #0.19
            return X_res
    
    def __back_conv_v2_3d__(X,w,stride,pad,grad):
        X_val,w_val  = Convolution2D.__get_input_value__(X,w)
        X_val        = Convolution2D.__pad__( X_val ,pad)
        x_shp,w_shp  = Convolution2D.__get_shape__(X_val,w_val)
        res_shape    = Convolution2D. __assert_shape__(x_shp,w_shp,stride)
        X_rev        = np.zeros(shape = x_shp)
        w_rev        = np.zeros(shape = w_shp)
        if res_shape:
            for k in range(w_shp[0]):
                for i in range(w_shp[2]):
                    for j in range(w_shp[3]):
                        for l in range(w_shp[1]):
                            row_idx = np.arange(i,x_shp[2]-w_shp[2]+i+1,stride)
                            col_idx = np.arange(j,x_shp[3]-w_shp[3]+j+1,stride)
                            w_rev[k,l,i,j] = np.sum(grad[:,k,:,:]*X_val[:,l,row_idx[:, None], col_idx])
                            X_rev[:,l,row_idx[:, None], col_idx] += grad[:,k,:,:]*w_val[k,l,i,j]
            return X_rev,w_rev
   

    def __conv_v5_3d__(X,w,stride,pad):
        t0 = time.time()
        X_val,w_val  = Convolution2D.__get_input_value__(X,w)
        X_val        = Convolution2D.__pad__( X_val ,pad)
        x_shp,w_shp  = Convolution2D.__get_shape__(X_val,w_val)
        res_shp    = Convolution2D. __assert_shape__(x_shp,w_shp,stride)
        X_res        = np.zeros(shape = (x_shp[0]*res_shp[2]*res_shp[3],w_shp[2]*w_shp[3]*w_shp[1]))
        t1 = time.time()
        print('Preliminary',t1-t0)
        if res_shp:
            
            """
            iter = -1
            for l in range(w_shp[1]):
                for i in range(w_shp[2]):
                    for j in range(w_shp[3]):

                        iter+=1
                        row_idx = np.arange(i,x_shp[2]-w_shp[2]+i+1,stride)
                        col_idx = np.arange(j,x_shp[3]-w_shp[3]+j+1,stride)

                   
                        X_res[:,:,iter] = np.reshape(X_val[:,l,row_idx[:, None], col_idx],(x_shp[0],res_shp[2]*res_shp[3]))
            """
            t0 = time.time()
            
            """uncomment im2col4 for code of the blog's author, uncomment im2col6 for CS231N course code"""
            X_res = im2col4.im2col(np.array(w_shp),np.array(x_shp),np.array(res_shp),X_val,X_res,stride)
            """
            X_res = im2col6.im2col_cython(X_val,w_shp[2],w_shp[3], 0,stride)
            X_res = X_res.T
            """
            t1 = time.time()
            print('Cython way im2col:',t1-t0)
            
            print(X_res.shape,(w_shp[1]*w_shp[2]*w_shp[3],w_shp[0]))
            t0 = time.time()
            #res=  np.dot(X_res.reshape(res_shp[2]*res_shp[3]*x_shp[0],w_shp[1]*w_shp[2]*w_shp[3]),w_val.reshape(w_shp[1]*w_shp[2]*w_shp[3],w_shp[0])).reshape(x_shp[0],res_shp[2]*res_shp[3],w_shp[0])
            res=  np.dot(X_res,w_val.reshape(w_shp[1]*w_shp[2]*w_shp[3],w_shp[0]))
            t1 = time.time()
            print('Dot product:',t1-t0)
            #res=res.reshape(x_shp[0],res_shp[2]*res_shp[3],w_shp[0])
            
            t0 = time.time()
            #Res = res.swapaxes(1,2).reshape(res_shp)
            Res = res.reshape(res_shp)
            t1 = time.time()
            print('Final Swapping,reshaping:',t1-t0)
            
            return Res
    def __back_conv_v5_3d__(X,w,stride,pad,grad):
        
        X_val,w_val  = Convolution2D.__get_input_value__(X,w)
        X_val        = Convolution2D.__pad__( X_val ,pad)
        x_shp,w_shp  = Convolution2D.__get_shape__(X_val,w_val)
        res_shp    = Convolution2D. __assert_shape__(x_shp,w_shp,stride)
        X_res        = np.zeros(shape = (x_shp[0],res_shp[2]*res_shp[3],w_shp[2]*w_shp[3]*w_shp[1]))
        if res_shp:
           
            indices = np.arange(w_shp[1])*(w_shp[2]*w_shp[3])
            iter = -1
            """
            for i in range(w_shp[2]):
                for j in range(w_shp[3]):

                    iter+=1
                    row_idx = np.arange(i,x_shp[2]-w_shp[2]+i+1,stride)
                    col_idx = np.arange(j,x_shp[3]-w_shp[3]+j+1,stride)
                    X_res[:,iter+indices,:] = np.reshape(X_val[:,:,row_idx[:, None], col_idx],(x_shp[0],w_shp[1],res_shp[2]*res_shp[3]))
            """
            t0 = time.time()
            X_res = im2col3.im2col(np.array(w_shp),np.array(x_shp),np.array(res_shp),X_val,X_res,stride)
            t1 = time.time()
            print('Cython way im2col',t1-t0)
            
            t0 = time.time()
            grad_reshaped = grad.reshape(x_shp[0],w_shp[0],res_shp[2]*res_shp[3]).swapaxes(1,2)
            t1 = time.time()
            print('grad reshaping',t1-t0)
            print(grad_reshaped.shape,(w_shp[0],w_shp[1]*w_shp[2]*w_shp[3]))
            t0 = time.time()
            w_val_res = w_val.reshape(w_shp[0],w_shp[1]*w_shp[2]*w_shp[3])
            t1 = time.time()
            print('W_val reshape',t1-t0)
            t0 = time.time()
            x_grad = np.dot(grad_reshaped, w_val_res)
            t1 = time.time()
            
            print('Dot prod for xgrad',t1-t0)
            
            t0 = time.time()
            x_grad = x_grad.swapaxes(1,2)#.reshape(x_shp[0],w_shp[1]*w_shp[2]*w_shp[3],res_shp[2]*res_shp[3])
            t1 = time.time()
            print('Swap axes',t1-t0)
            
            X_grad = np.zeros(x_shp)
            
            
            """
            t0 = time.time()
            iter = -1 
            for l in range(w_shp[1]):
                for i in range(w_shp[2]):
                    for j in range(w_shp[3]):

                        iter+=1
                        row_idx = np.arange(i,x_shp[2]-w_shp[2]+i+1,stride)
                        col_idx = np.arange(j,x_shp[3]-w_shp[3]+j+1,stride)

                        X_grad[:,l,row_idx[:, None], col_idx] += x_grad[:,iter,:].reshape(x_shp[0],res_shp[2],res_shp[3]) 
           
            t1 = time.time()
            print('Python way',t1-t0)
            """
            t0 = time.time()
            X_grad = col2im.col2im(np.array(w_shp),np.array(x_shp),np.array(res_shp),X_grad,x_grad,stride)
            t1 = time.time()
            print('Cython way col2im',t1-t0)
            
            t0 = time.time()
            grad=grad_reshaped.swapaxes(1,2).swapaxes(0,1).reshape(w_shp[0],res_shp[2]*res_shp[3]*x_shp[0])
            X_res= X_res.reshape(res_shp[2]*res_shp[3]*x_shp[0],w_shp[1]*w_shp[2]*w_shp[3])
            t1 = time.time()
            print('Reshaping',t1-t0)
            
            t0 = time.time()
            w_grad = np.dot(grad,X_res)
            t1 = time.time()
            print('Dot for finding w_grad',t1-t0)
            
            w_grad = w_grad.reshape(w_shp)
            return(X_grad,w_grad)
      
            
            

In [12]:
# Testing and benchmarking backprop
import numpy as np
import time 
np.random.rand(1)
xval = np.random.rand(100, 10, 31, 31)
wval = np.random.rand(25, 10, 5,5)
x1 = Tensor(xval)
w1 = Tensor(wval)
t0 = time.time()
Res1 = Convolution2D.__conv_v5_3d__(x1,w1,stride=2,pad=0)
t1 = time.time()
print('Forward_prop',t1-t0 )
"""
t0 = time.time()
Res2 = Convolution2D.__conv_v2_3d__(x1,w1,stride=2,pad=0)
t1 = time.time()
print('Forward_prop',t1-t0 )

assert(abs(Res1-Res2)<10**-7).all()
"""
"""
t0 = time.time()
bk1 = Convolution2D.__back_conv_v2_3d__(x1,w1,stride=2,pad=0,grad=Res1)

t1 = time.time()
print('Back_prop',t1-t0 )
"""
t0 = time.time()
bk2 = Convolution2D.__back_conv_v5_3d__(x1,w1,stride=2,pad=0,grad=Res1)
t1 = time.time()
print('Back_prop',t1-t0) 
#"""
#assert(abs(bk2[0]-bk1[0])<10**-9).all()

Preliminary 0.010974407196044922
Cython way im2col: 0.03797578811645508
(19600, 250) (250, 25)
Dot product: 0.03198385238647461
Final Swapping,reshaping: 0.0
Forward_prop 0.0869302749633789
Cython way im2col 0.04297590255737305
grad reshaping 0.0
(100, 196, 25) (25, 250)
W_val reshape 0.0
Dot prod for xgrad 0.16290688514709473
Swap axes 0.0
Cython way col2im 0.06229829788208008
Reshaping 0.002022266387939453
Dot for finding w_grad 0.013972043991088867
Back_prop 0.30616116523742676
